In [1]:
import re
import os
import pymongo
import pprint
from utils import *
import multiprocessing as mp
from functools import partial

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'gutenberg_db'), 'gutenberg_collection')